# Users & groups

This page discusses ways to manage users in a Linux system.

Commands described in this page can seriously change the environment, so it is highly recommended to run this notebook from the safe environment.

## Users file

In a Linux system, the `/etc/passwd` file contains information about users, with each line representing a user. The information about each user is separated by colons (`:`). Lines describing a user follow this pattern:

```
<user name>:<password placeholder>:<user id>:<primary group id>:<user comment>:<home directory of the user>:<shell of the user>
```

---

The following cell shows the content that `/etc/passwd` can have.

In [7]:
cat /etc/passwd | head -n 10

root:x:0:0:root:/root:/bin/bash
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
man:x:6:12:man:/var/cache/man:/usr/sbin/nologin
lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin
mail:x:8:8:mail:/var/mail:/usr/sbin/nologin
news:x:9:9:news:/var/spool/news:/usr/sbin/nologin


## Groups file

There is a `/etc/group` file where each line contains information about a group. Each line in `/etc/group` file corresponds to pattern:

```
<group name>:<password placeholder>:<group id>:<users list>
```

---

The following cell shows typical `/etc/group` file. 

In [5]:
cat /etc/group | head -n 10

root:x:0:
daemon:x:1:
bin:x:2:
sys:x:3:
adm:x:4:
tty:x:5:
disk:x:6:
lp:x:7:
mail:x:8:
news:x:9:


## Passwords file

File that contains passwords is `/etc/shadow`. The meaning of the fileds of this file is:

```
<user name>:<password hash or password setting>:<last password change>:<minimum days before change>:<maximum days before expiration>:<warning period>:<inactive period>:<expires on>
```

In [8]:
cat /etc/shadow

root:*:20101:0:99999:7:::
daemon:*:20101:0:99999:7:::
bin:*:20101:0:99999:7:::
sys:*:20101:0:99999:7:::
sync:*:20101:0:99999:7:::
games:*:20101:0:99999:7:::
man:*:20101:0:99999:7:::
lp:*:20101:0:99999:7:::
mail:*:20101:0:99999:7:::
news:*:20101:0:99999:7:::
uucp:*:20101:0:99999:7:::
proxy:*:20101:0:99999:7:::
www-data:*:20101:0:99999:7:::
backup:*:20101:0:99999:7:::
list:*:20101:0:99999:7:::
irc:*:20101:0:99999:7:::
_apt:*:20101:0:99999:7:::
nobody:*:20101:0:99999:7:::


## Primary groups

A group can be primary or regular for a user. Each user has one and only one primary group. A user can belong to multiple regular groups.

- The primary group is specified in the user's entry in `/etc/passwd` in the fourth field, which contains the group's GID (Group ID).
- Regular (non-primary) user-group relationships are defined in `/etc/group` in the last field - each group lists the additional users who are members of that group.

---

The following cell recreates the `test_user` that would be used as an example. And shows the record of the `/etc/passwd` that corresponds to that user.

In [43]:
userdel test_user &> /dev/null
groupdel test_user &> /dev/null
useradd test_user
cat /etc/passwd | grep test_user

test_user:x:1003:1004::/home/test_user:/bin/sh


In the field corresponding to the primary group there is a number. The next cell checks if there is anything in `/etc/group` that corresponds to the `test_user`.

In [45]:
cat /etc/group | grep test_user

test_user:x:1004:


There is automatically created group with the same name and index as specified in the `/etc/passwd`.

To show the difference the following cell creates `test_group` and adds `test_user` to it.

In [58]:
groupdel test_group &> /dev/null
groupadd test_group
gpasswd -a test_user test_group

Adding user test_user to group test_group


In case of `test_group`, the user is specified in the last filed describing the group.

In [59]:
cat /etc/group | grep test_group

test_group:x:1005:test_user


## Add user

This section discusses features associated with adding extra users to the system. The parameters of the `useradd` command are primarily addressed here.

### Default shell

You can specify default shell that will be used by the user. With `--shell` argument.

---

The following example shows how to make python the default shell for the user. We need python installed on the system to play with such tricks.

The following cell creates `python_user` and specifies the path to Python in the `--shell` argument.

In [1]:
useradd --shell /usr/bin/python3 -m python_user

The following cell executes Python code that prints the value of the environment variable `SHELL` for `python_user`.

In [2]:
su - python_user -c "import os; print(os.environ['SHELL'])"

/usr/bin/python3


As a result, `/usr/bin/python` is shown, and Python syntax is used to print it.

## Add/remove group

Use the `groupadd` and `groupdel` commands to add and remove groups.

---

The following code creates `newgroup` and shows the corresponding line in `/etc/group`.

In [13]:
groupadd newgroup
cat /etc/group | grep newgrou

newgroup:x:1000:


But after applying `groupdel` to the created group, it disappears from the `/etc/group`.

In [14]:
groupdel newgroup
cat /etc/group | grep newgroup | true

## Configure relationships

This section covers commands that allow you to manage relationships between users and groups, such as setting information about which users belong to which groups.

Really basics:

- Add user to group with `gpasswd -a <user name> <group name>`.
- Delete user from group with `gpasswd -d <user name> <group name>`.

---

The following cell creates an `experimental` group, which will be used as an example:

In [34]:
groupdel experimental &> /dev/null && groupadd experimental
cat /etc/group | grep experimental

experimental:x:1000:


By default, there are no users in this group. However, following cell with the syntax `usermod -a <username> experimental` will add two users to the group.

In [35]:
gpasswd -a root experimental
gpasswd -a bin experimental
cat /etc/group | grep experimental

Adding user root to group experimental
Adding user bin to group experimental
experimental:x:1000:root,bin


As the result, line of the `/etc/group` corresponding to the `experimental` contains users separated by coma.

The following cell removes the `root` user from the `experimental` group. And shows the corresponding line of the `/etc/group` - now there is no `root` user.

In [36]:
gpasswd -d root experimental
cat /etc/group | grep experimental

Removing user root from group experimental


experimental:x:1000:bin


### Primary group

You can change primary group of the user by `usermode -g <group name> <user name>`.

---

The following cell recreates the `group_example` user, which we'll use as an example.

In [33]:
userdel group_example &> /dev/null
groupdel group_example &> /dev/null
useradd group_example

The next cell shows the line in `/etc/passwd` that corresponds to the user under consideration. The index of the primary group is the fourth field.

In [34]:
cat /etc/passwd | grep group_example

group_example:x:1001:1001::/home/group_example:/bin/sh


The following cell changes the primary group of the `group_example`. And shows the line in the configuration file.

In [35]:
usermod -g root group_example
cat /etc/passwd | grep group_example

group_example:x:1001:0::/home/group_example:/bin/sh


To be sure, let's check if this id really belongs to the `root` user.

In [37]:
cat /etc/group | grep root

root:x:0:
